---
title: "Hurtig Start med DuckDB i Python"

subtitle: "Dataanalyse i DuckDB for Python-brugere"
order: 1

format: 
    html:
         x: y
        #  order: 1
    revealjs:
        x: z
        output-file: DuckDB_quickstart_slide.html


---


# Installation med pip

Du behøver faktisk ikke installere en server, eller et decideret commando-linje-program, selv om man godt kan.

Du kan bare nøjes med at installere en python-pakke, med `pip`.

## Installer DuckDB med pip

I _cmd_ / _PowerShell_ / _bash_:

```cmd
pip install duckdb
```

I _python_:

In [ ]:
#| echo: true

# Kontroller installation
import duckdb
print(f"DuckDB version: {duckdb.__version__}")

# Importer pandas for DataFrame visning
import pandas as pd

## Tilslutning til database


In [ ]:
#| echo: true

# Opret forbindelse til in-memory database
conn = duckdb.connect()

# ELLER tilslut til eksisterende fil-database
conn = duckdb.connect('demo.db')

# Alternativt, download en demo-database
# conn.execute("INSTALL httpfs; LOAD httpfs;")
# conn.execute("CREATE TABLE demo AS SELECT * FROM 'https://example.com/demodata.csv'")
conn.close()

## Første `SELECT` og visning som DataFrame


In [ ]:
#| echo: false
#| output: false

# connect til in-memory-database, for reste af denne tekst
conn = duckdb.connect()

In [ ]:
#| echo: true
conn = duckdb.connect()
# Opret en simpel test-tabel for demo
conn.execute("CREATE TABLE test(id INTEGER, navn VARCHAR)")
conn.execute("INSERT INTO test VALUES (1, 'Anders'), (2, 'Mette'), (3, 'Lars')")

# Kør en SELECT og vis som DataFrame
result_df = conn.execute("SELECT * FROM test").df()

# Vis DataFrame
print(result_df)

<!-- ```
   id    navn
0   1  Anders
1   2   Mette
2   3    Lars
``` -->

## Indsæt og opdater data

In [ ]:
#| echo: true
# Indsæt en enkelt række
conn.execute("INSERT INTO test VALUES (4, 'Sofie')")

# Indsæt fra Python variabler
person_id = 5
navn = "Peter"
conn.execute("INSERT INTO test VALUES (?, ?)", [person_id, navn])

# Opdater data
conn.execute("UPDATE test SET navn = 'Anders Nielsen' WHERE id = 1")

# Kontroller ændringerne
print(conn.execute("SELECT * FROM test").df())

## Opret ny tabel og indsæt data


In [ ]:
#| echo: false
#| output: false

# clean up before CREATE 

# conn = duckdb.connect('demo.db')
# conn.execute("DROP TABLE IF EXISTS produkter")
""

In [ ]:
#| echo: true
#| output: false

# Opret tabel med constraints
conn.execute("""
CREATE TABLE produkter (
    id INTEGER PRIMARY KEY,
    navn VARCHAR NOT NULL,
    pris DECIMAL(10,2),
    kategori VARCHAR
)
""")

# Bulk indsættelse af data
conn.execute("""
INSERT INTO produkter VALUES
    (1, 'Laptop', 5999.99, 'Elektronik'),
    (2, 'Skrivebord', 1499.00, 'Møbler'),
    (3, 'Tastatur', 499.50, 'Elektronik'),
    (4, 'Stol', 899.00, 'Møbler')
""")

## Analyse


In [ ]:
#| echo: true
# Foretag analyse med SQL
result = conn.execute("""
    SELECT 
        kategori, 
        COUNT(*) as antal, 
        AVG(pris) as gennemsnit_pris,
        SUM(pris) as total_pris
    FROM produkter
    GROUP BY kategori
""").df()

print(result)

## Visualisering


In [ ]:
#| echo: true
# Fra DuckDB til pandas til visualisering
import matplotlib.pyplot as plt

result.plot(kind='bar', x='kategori', y='total_pris')
plt.title('Total pris per kategori')
plt.ylabel('Total pris (DKK)')
plt.tight_layout()
plt.show()